In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torchvision

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mnist-in-csv/mnist_test.csv
/kaggle/input/mnist-in-csv/mnist_train.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
%matplotlib inline

In [3]:
data_train = pd.read_csv("/kaggle/input/mnist-in-csv/mnist_train.csv")
data_test = pd.read_csv("/kaggle/input/mnist-in-csv/mnist_test.csv")
features = data_train.columns[data_train.columns != 'label']
data_train

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# tsne = TSNE(n_components=2, random_state=1)
# visualize = tsne.fit_transform(data_train[features])

# plt.figure(figsize = (30,12))
# plt.subplot(122)
# plt.scatter(X_reduced[:,0],X_reduced[:,1],  c = df_train[target], 
#             cmap = "coolwarm", edgecolor = "None", alpha=0.35)
# plt.colorbar()
# plt.title('TSNE Scatter Plot')
# plt.show()

In [5]:
input_size = 784
epochs = 10
hidden_layers=500
num_classes = 10
batch_size = 100
learning_rate = 0.001 

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_layers, num_classes):
        super(NeuralNetwork, self).__init__()
        self.input_size = input_size
        self.l1= nn.Linear(input_size, hidden_layers)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_layers, num_classes)  
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out 

model = NeuralNetwork(input_size, hidden_layers, num_classes)

In [7]:
def training_model(training_set, model, epochs, testing_set):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
    total_steps = len(training_set)
    
    for epoch in range(epochs):
        for i in range(0, len(training_set), batch_size):
            images_batch = training_set[features][i:i+batch_size]
            images_batch = torch.from_numpy(np.array(images_batch)).float()
            labels_batch = training_set.label[i:i+batch_size]
            labels_batch = torch.from_numpy(np.array(labels_batch))
            
            # Forward propogation
            outputs = model(images_batch)
            loss = criterion(outputs, labels_batch)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step() 
            
        
        print (f'Epoch [{epoch+1}/{epochs}], Step[{i+1}/{total_steps}], Loss: {loss.item():.4f}')
        testing_model(model, testing_set)

In [8]:
def testing_model(model, training_set):
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for i in range(0, len(training_set), batch_size):
            images_batch = training_set[features][i:i+batch_size]
            images_batch = torch.from_numpy(np.array(images_batch)).float()
            labels_batch = training_set.label[i:i+batch_size]
            labels_batch = torch.from_numpy(np.array(labels_batch))
            
         
            outputs = model(images_batch)

            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels_batch.size(0)
            n_correct += (predicted == labels_batch).sum().item()

        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy of the network on the 10000 test images: {acc} %')

In [9]:
training_model(data_train, model, epochs, data_test)

Epoch [1/10], Step[59901/60000], Loss: 0.3631
Accuracy of the network on the 10000 test images: 93.91 %
Epoch [2/10], Step[59901/60000], Loss: 0.2637
Accuracy of the network on the 10000 test images: 95.09 %
Epoch [3/10], Step[59901/60000], Loss: 0.2245
Accuracy of the network on the 10000 test images: 93.66 %
Epoch [4/10], Step[59901/60000], Loss: 0.1779
Accuracy of the network on the 10000 test images: 94.59 %
Epoch [5/10], Step[59901/60000], Loss: 0.2512
Accuracy of the network on the 10000 test images: 95.56 %
Epoch [6/10], Step[59901/60000], Loss: 0.1340
Accuracy of the network on the 10000 test images: 94.52 %
Epoch [7/10], Step[59901/60000], Loss: 0.2165
Accuracy of the network on the 10000 test images: 95.71 %
Epoch [8/10], Step[59901/60000], Loss: 0.0862
Accuracy of the network on the 10000 test images: 95.88 %
Epoch [9/10], Step[59901/60000], Loss: 0.2216
Accuracy of the network on the 10000 test images: 96.15 %
Epoch [10/10], Step[59901/60000], Loss: 0.0867
Accuracy of the n